<h2>Enrichment and transformations<h2>

<h3>read the JSON<h3>

In [1]:
import pandas as pd
import numpy as np
import requests

df = pd.read_json('auto.json', orient='records')
df.set_index('CarNumber', inplace=True)
pd.options.display.float_format = '${:,.2f}'.format
np.random.seed(21)
df

,Refund,Fines,Make,Model
CarNumber,,,,
Y163O8161RUS,2,"$3,200.00",Ford,Focus
E432XX77RUS,1,"$6,500.00",Toyota,Camry
7184TT36RUS,1,"$2,100.00",Ford,Focus
X582HE161RUS,2,"$2,000.00",Ford,Focus
92918M178RUS,1,"$5,700.00",Ford,Focus
...,...,...,...,...
Y163O8161RUS,2,"$1,600.00",Ford,Focus
M0309X197RUS,1,"$22,300.00",Ford,Focus
O673E8197RUS,2,$600.00,Ford,Focus


<h3>enrich the dataframe<h3>

In [2]:
sample = df.sample(200, random_state=21)
sample

,Refund,Fines,Make,Model
CarNumber,,,,
M0299X197RUS,2,"$19,200.00",Ford,Focus
83298C154RUS,2,"$8,594.59",Ford,Focus
H957HY161RUS,1,"$2,000.00",Ford,Focus
T941CC96RUS,1,"$2,000.00",Ford,Focus
H966HY161RUS,1,$500.00,Ford,Focus
...,...,...,...,...
8182XX154RUS,1,$200.00,Ford,Focus
X796TH96RUS,1,$500.00,Ford,Focus
T011MY163RUS,2,"$4,000.00",Ford,Focus


In [3]:
sample.Refund = np.random.randint(1,3, size=sample.shape[0])
sample.Fines = np.random.uniform(df.Fines.min(),df.Fines.max(), size=sample.shape[0])
sample

,Refund,Fines,Make,Model
CarNumber,,,,
M0299X197RUS,2,"$124,324.87",Ford,Focus
83298C154RUS,2,"$140,286.47",Ford,Focus
H957HY161RUS,1,"$8,596.52",Ford,Focus
T941CC96RUS,1,"$13,513.56",Ford,Focus
H966HY161RUS,1,"$108,911.53",Ford,Focus
...,...,...,...,...
8182XX154RUS,1,"$158,871.09",Ford,Focus
X796TH96RUS,1,"$68,304.32",Ford,Focus
T011MY163RUS,2,"$91,342.35",Ford,Focus


In [4]:
concat_rows = pd.concat([df, sample])
concat_rows

,Refund,Fines,Make,Model
CarNumber,,,,
Y163O8161RUS,2,"$3,200.00",Ford,Focus
E432XX77RUS,1,"$6,500.00",Toyota,Camry
7184TT36RUS,1,"$2,100.00",Ford,Focus
X582HE161RUS,2,"$2,000.00",Ford,Focus
92918M178RUS,1,"$5,700.00",Ford,Focus
...,...,...,...,...
8182XX154RUS,1,"$158,871.09",Ford,Focus
X796TH96RUS,1,"$68,304.32",Ford,Focus
T011MY163RUS,2,"$91,342.35",Ford,Focus


<h3>enrich the dataframe concat_rows by a new column with the data generated<h3>

In [5]:
years = pd.Series(np.random.randint(1980, 2020, size=concat_rows.shape[0]), name='Year', index=concat_rows.index)
fines = pd.concat([concat_rows, years], axis=1)
fines

,Refund,Fines,Make,Model,Year
CarNumber,,,,,
Y163O8161RUS,2,"$3,200.00",Ford,Focus,2017
E432XX77RUS,1,"$6,500.00",Toyota,Camry,1990
7184TT36RUS,1,"$2,100.00",Ford,Focus,1995
X582HE161RUS,2,"$2,000.00",Ford,Focus,1989
92918M178RUS,1,"$5,700.00",Ford,Focus,1982
...,...,...,...,...,...
8182XX154RUS,1,"$158,871.09",Ford,Focus,2017
X796TH96RUS,1,"$68,304.32",Ford,Focus,2017
T011MY163RUS,2,"$91,342.35",Ford,Focus,1994


<h3>enrich the dataframe with the data from another dataframe<h3>

<h4>create a new dataframe with the car numbers and their owners<h4>

In [6]:
surnames = pd.read_json('surname.json')
surnames.columns = surnames.iloc[0].values
surnames.drop(surnames.index[0], axis=0, inplace=True)
surnames

,NAME,COUNT,RANK
1,ADAMS,427865,42
2,ALLEN,482607,33
3,ALVAREZ,233983,92
4,ANDERSON,784404,15
5,BAILEY,277845,72
...,...,...,...
96,WILLIAMS,1625252,3
97,WILSON,801882,14
98,WOOD,250715,84
99,WRIGHT,458980,35


In [8]:
surnames_sample = surnames.sample(fines.index.unique().shape[0], random_state=21, replace=True)['NAME']
surnames_sample

74    RICHARDSON
80          ROSS
57        MORGAN
5         BAILEY
49         LOPEZ
         ...    
10      CAMPBELL
32          HALL
6          BAKER
21          DIAZ
57        MORGAN
Name: NAME, Length: 531, dtype: object

In [9]:
owners = pd.DataFrame(surnames_sample)
owners.index = fines.index.unique()
owners.columns = ['SURNAME']
owners

,SURNAME
CarNumber,
Y163O8161RUS,RICHARDSON
E432XX77RUS,ROSS
7184TT36RUS,MORGAN
X582HE161RUS,BAILEY
92918M178RUS,LOPEZ
...,...
O136HO197RUS,CAMPBELL
O22097197RUS,HALL
M0309X197RUS,BAKER


<h3>append 5 more observations to the fines dataframe<h3>

In [10]:
fines.groupby('CarNumber').first()

,Refund,Fines,Make,Model,Year
CarNumber,,,,,
704687163RUS,2,"$1,400.00",Ford,Focus,1992
704787163RUS,2,"$2,800.00",Ford,Focus,1988
704987163RUS,2,"$8,594.59",Ford,Focus,2008
705287163RUS,2,"$2,000.00",Ford,Focus,2000
705387163RUS,2,$700.00,Ford,Focus,2019
...,...,...,...,...,...
Y965O8197RUS,2,"$1,600.00",Ford,Focus,1987
Y966O8197RUS,2,"$10,400.00",Ford,Focus,1998
Y967O8197RUS,1,"$4,000.00",Ford,Focus,1996


In [11]:
new_refund = np.random.randint(1,3)
new_fines = np.random.uniform(fines.Fines.min(), fines.Fines.max())
i = np.random.randint(1, fines.index.unique().shape[0])
new_make = fines.Make[i]
new_model = fines.Model[i]
new_year = fines.Year[i]

new_obs = [['704887163RUS', new_refund, new_fines, new_make, new_model, new_year],
            ['705087163RUS', new_refund, new_fines, new_make, new_model, new_year],
            ['705187163RUS', new_refund, new_fines, new_make, new_model, new_year],
            ['Y970O8197RUS', new_refund, new_fines, new_make, new_model, new_year],
            ['Y971O8197RUS', new_refund, new_fines, new_make, new_model, new_year]]
new_obs = pd.DataFrame(new_obs, columns=fines.reset_index().columns)
new_obs.set_index('CarNumber', inplace=True)
new_obs

,Refund,Fines,Make,Model,Year
CarNumber,,,,,
704887163RUS,2,"$163,151.05",Ford,Focus,2014
705087163RUS,2,"$163,151.05",Ford,Focus,2014
705187163RUS,2,"$163,151.05",Ford,Focus,2014
Y970O8197RUS,2,"$163,151.05",Ford,Focus,2014
Y971O8197RUS,2,"$163,151.05",Ford,Focus,2014


In [12]:
fines = pd.concat([fines, new_obs])
fines

,Refund,Fines,Make,Model,Year
CarNumber,,,,,
Y163O8161RUS,2,"$3,200.00",Ford,Focus,2017
E432XX77RUS,1,"$6,500.00",Toyota,Camry,1990
7184TT36RUS,1,"$2,100.00",Ford,Focus,1995
X582HE161RUS,2,"$2,000.00",Ford,Focus,1989
92918M178RUS,1,"$5,700.00",Ford,Focus,1982
...,...,...,...,...,...
704887163RUS,2,"$163,151.05",Ford,Focus,2014
705087163RUS,2,"$163,151.05",Ford,Focus,2014
705187163RUS,2,"$163,151.05",Ford,Focus,2014


<h3>delete the dataframe last 20 observations from the owners and add 3 new observations<h3>

In [13]:
owners.drop(owners.tail(20).index, inplace=True)
new_own = [['Y972O8197RUS', 'WILLIAMS'],
            ['Y974O8197RUS', 'WILSON'],
            ['Y975O8197RUS', 'WOOD']]
new_own = pd.DataFrame(new_own, columns=owners.reset_index().columns)
new_own.set_index('CarNumber', inplace=True)
owners = pd.concat([owners, new_own])
owners

,SURNAME
CarNumber,
Y163O8161RUS,RICHARDSON
E432XX77RUS,ROSS
7184TT36RUS,MORGAN
X582HE161RUS,BAILEY
92918M178RUS,LOPEZ
...,...
O50197197RUS,WRIGHT
7608EE777RUS,HILL
Y972O8197RUS,WILLIAMS


<h3>join both dataframes<h3>

<h4>the new dataframe should have only the car numbers that exist in both dataframes<h4>

In [14]:
pd.merge(fines, owners, how='inner', left_index=True, right_on='CarNumber')

,Refund,Fines,Make,Model,Year,SURNAME
CarNumber,,,,,,
Y163O8161RUS,2,"$3,200.00",Ford,Focus,2017,RICHARDSON
Y163O8161RUS,2,"$1,600.00",Ford,Focus,1996,RICHARDSON
E432XX77RUS,1,"$6,500.00",Toyota,Camry,1990,ROSS
E432XX77RUS,2,"$13,000.00",Toyota,Camry,2019,ROSS
7184TT36RUS,1,"$2,100.00",Ford,Focus,1995,MORGAN
...,...,...,...,...,...,...
E41977152RUS,2,"$2,400.00",Ford,Focus,2009,BAKER
9464EX178RUS,2,"$2,100.00",Ford,Focus,2001,MARTIN
O50197197RUS,2,"$7,800.00",Ford,Focus,2018,WRIGHT


<h4>the new dataframe should have all the car numbers that exist in both
dataframes<h4>

In [15]:
pd.merge(fines, owners, how='outer', left_index=True, right_on='CarNumber')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
Y163O8161RUS,Y163O8161RUS,$2.00,"$3,200.00",Ford,Focus,"$2,017.00",RICHARDSON
Y163O8161RUS,Y163O8161RUS,$2.00,"$1,600.00",Ford,Focus,"$1,996.00",RICHARDSON
E432XX77RUS,E432XX77RUS,$1.00,"$6,500.00",Toyota,Camry,"$1,990.00",ROSS
E432XX77RUS,E432XX77RUS,$2.00,"$13,000.00",Toyota,Camry,"$2,019.00",ROSS
7184TT36RUS,7184TT36RUS,$1.00,"$2,100.00",Ford,Focus,"$1,995.00",MORGAN
...,...,...,...,...,...,...,...
NaN,Y970O8197RUS,$2.00,"$163,151.05",Ford,Focus,"$2,014.00",NaN
NaN,Y971O8197RUS,$2.00,"$163,151.05",Ford,Focus,"$2,014.00",NaN
Y972O8197RUS,Y972O8197RUS,$nan,$nan,NaN,NaN,$nan,WILLIAMS
Y974O8197RUS,Y974O8197RUS,$nan,$nan,NaN,NaN,$nan,WILSON


<h4>the new dataframe should have only the car numbers from the fines dataframe<h4>

In [16]:
pd.merge(fines, owners, how='left', left_index=True, right_on='CarNumber')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
Y163O8161RUS,Y163O8161RUS,2,"$3,200.00",Ford,Focus,2017,RICHARDSON
E432XX77RUS,E432XX77RUS,1,"$6,500.00",Toyota,Camry,1990,ROSS
7184TT36RUS,7184TT36RUS,1,"$2,100.00",Ford,Focus,1995,MORGAN
X582HE161RUS,X582HE161RUS,2,"$2,000.00",Ford,Focus,1989,BAILEY
92918M178RUS,92918M178RUS,1,"$5,700.00",Ford,Focus,1982,LOPEZ
...,...,...,...,...,...,...,...
NaN,704887163RUS,2,"$163,151.05",Ford,Focus,2014,NaN
NaN,705087163RUS,2,"$163,151.05",Ford,Focus,2014,NaN
NaN,705187163RUS,2,"$163,151.05",Ford,Focus,2014,NaN
NaN,Y970O8197RUS,2,"$163,151.05",Ford,Focus,2014,NaN


<h4>the new dataframe should have only the car numbers from the owners dataframe<h4>

In [17]:
pd.merge(fines, owners, how='right', left_index=True, right_on='CarNumber')

,Refund,Fines,Make,Model,Year,SURNAME
CarNumber,,,,,,
Y163O8161RUS,$2.00,"$3,200.00",Ford,Focus,"$2,017.00",RICHARDSON
Y163O8161RUS,$2.00,"$1,600.00",Ford,Focus,"$1,996.00",RICHARDSON
E432XX77RUS,$1.00,"$6,500.00",Toyota,Camry,"$1,990.00",ROSS
E432XX77RUS,$2.00,"$13,000.00",Toyota,Camry,"$2,019.00",ROSS
7184TT36RUS,$1.00,"$2,100.00",Ford,Focus,"$1,995.00",MORGAN
...,...,...,...,...,...,...
7608EE777RUS,$1.00,"$4,000.00",Skoda,Octavia,"$2,019.00",HILL
7608EE777RUS,$2.00,"$158,380.52",Skoda,Octavia,"$1,994.00",HILL
Y972O8197RUS,$nan,$nan,NaN,NaN,$nan,WILLIAMS


<h3>create a pivot table from the fines dataframe<h3>

In [18]:
pd.pivot_table(fines, index=['Make', 'Model'], values='Fines', columns='Year')

Year                     1980       1981       1982       1983       1984  \
Make       Model                                                            
Ford       Focus   $41,849.71 $34,627.84 $41,290.03 $31,043.34 $14,928.26   
           Mondeo        $nan       $nan       $nan       $nan       $nan   
Skoda      Octavia $54,524.98       $nan  $5,148.65       $nan       $nan   
Toyota     Camry         $nan       $nan       $nan       $nan  $7,500.00   
           Corolla $41,261.40  $8,594.59       $nan  $6,800.00  $7,800.00   
Volkswagen Golf          $nan  $5,000.00  $9,300.00       $nan       $nan   
           Jetta         $nan       $nan       $nan       $nan $86,818.87   
           Passat  $12,800.00  $2,100.00       $nan $19,147.29       $nan   
           Touareg  $1,300.00       $nan       $nan       $nan       $nan   

Year                     1985        1986       1987       1988        1989  \
Make       Model                                                              
Ford       Focus   $47,690.29  $37,672.20 $28,738.98 $27,683.95  $15,565.03   
           Mondeo   $6,700.00        $nan       $nan       $nan        $nan   
Skoda      Octavia    $400.00   $1,300.00 $21,869.25       $nan  $51,298.20   
Toyota     Camry    $1,000.00        $nan  $6,400.00    $500.00        $nan   
           Corolla       $nan     $900.00       $nan  $4,400.00        $nan   
Volkswagen Golf          $nan   $3,800.00       $nan $16,152.50  $10,600.00   
           Jetta         $nan   $4,000.00       $nan       $nan        $nan   
           Passat        $nan $178,793.38       $nan  $3,150.00 $122,452.12   
           Touareg       $nan        $nan       $nan       $nan        $nan   

Year                ...        2010       2011       2012       2013  \
Make       Model    ...                                                
Ford       Focus    ...  $22,702.55 $31,218.00 $24,802.82 $31,314.83   
           Mondeo   ...        $nan       $nan       $nan       $nan   
Skoda      Octavia  ...   $3,000.00 $21,764.86 $82,866.02    $500.00   
Toyota     Camry    ...   $1,000.00       $nan       $nan  $8,900.00   
           Corolla  ...        $nan       $nan       $nan $67,383.67   
Volkswagen Golf     ... $135,320.54  $5,800.00       $nan $56,284.29   
           Jetta    ...   $9,000.00       $nan       $nan       $nan   
           Passat   ...        $nan       $nan       $nan  $9,900.00   
           Touareg  ...        $nan       $nan       $nan       $nan   

Year                     2014       2015       2016        2017        2018  \
Make       Model                                                              
Ford       Focus   $58,959.69 $39,475.58 $17,425.60  $29,028.24  $21,139.58   
           Mondeo        $nan       $nan  $8,600.00  $34,400.00        $nan   
Skoda      Octavia       $nan  $4,797.29  $8,448.93  $63,300.00        $nan   
Toyota     Camry         $nan       $nan $70,675.66   $8,594.59  $19,800.00   
           Corolla       $nan       $nan       $nan   $7,600.00        $nan   
Volkswagen Golf          $nan $18,400.00       $nan        $nan $168,000.00   
           Jetta         $nan       $nan       $nan        $nan        $nan   
           Passat  $13,950.63 $10,450.00       $nan $143,229.68        $nan   
           Touareg       $nan       $nan       $nan        $nan        $nan   

Year                     2019  
Make       Model               
Ford       Focus   $26,947.14  
           Mondeo   $2,200.00  
Skoda      Octavia  $4,000.00  
Toyota     Camry   $13,000.00  
           Corolla       $nan  
Volkswagen Golf          $nan  
           Jetta         $nan  
           Passat        $nan  
           Touareg       $nan  

[9 rows x 40 columns]

<h3>save both the fines and owners dataframes to CSV files without an index<h3>

In [21]:
fines.reset_index().to_csv('fines.csv', index=False)
owners.reset_index().to_csv('owners.csv', index=False)